This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
url = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}'
params = {
    'start_date': '2020-05-29',
    'end_date': '2020-06-01'
}
r = requests.get(url, params)
r.status_code

200

In [4]:
response = r.json()

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-05-29',
  'end_date': '2020-05-29',
  'frequency': 'daily',
  'data': [['2020-05-29',
    91.05,
    93.1,
    90.25,
    92.1,
    None,
    238162.0,
    21887648.1,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [6]:
params = {
    'start_date': '2017-01-01',
    'end_date': '2017-12-31',
}
r = requests.get(url, params)
print(f"Status code: {r.status_code}")

Status code: 200


## 2. Convert the returned JSON object into a Python dictionary.

In [7]:
response = r.json()

In [8]:
data = response['dataset_data']['data']
column_names = response['dataset_data']['column_names']
print(f'There are {len(data)} dates.')
print(list(enumerate(column_names)))

There are 255 dates.
[(0, 'Date'), (1, 'Open'), (2, 'High'), (3, 'Low'), (4, 'Close'), (5, 'Change'), (6, 'Traded Volume'), (7, 'Turnover'), (8, 'Last Price of the Day'), (9, 'Daily Traded Units'), (10, 'Daily Turnover')]


## 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
# Start with the first opening price
highest_open_price = data[0][1]
lowest_open_price = data[0][1]

for row in data:
    open_price = row[1]
    if open_price:
        highest_open_price = max(open_price, highest_open_price)
        lowest_open_price = min(open_price, lowest_open_price)
    
print(f'The highest open price was {"${:.2f}".format(highest_open_price)}')
print(f'The lowest open price was {"${:.2f}".format(lowest_open_price)}')

The highest open price was $53.11
The lowest open price was $34.00


## 4. What was the largest change in any one day (based on High and Low price)?

In [10]:
# Start with the first row
largest_change_index = 0
largest_change = data[0][2] - data[0][3]

for index, row in enumerate(data):
    change = abs(row[2] - row[3])
    if change > largest_change:
        largest_change = change
        largest_change_index = index
    
print(f'The largest change was {"${:.2f}".format(data[largest_change_index][2] - data[largest_change_index][3])} on {data[largest_change_index][0]}.')

The largest change was $2.81 on 2017-05-11.


## 5. What was the largest change between any two days (based on Closing Price)?

In [11]:
# Start with the first row
largest_change_index = 0
largest_change = abs(data[0][4] - data[0][1])

for index, row in enumerate(data):
    if row[1]:
        change = abs(row[4] - row[1])
        if change > largest_change:
            largest_change = change
            largest_change_index = index

print(f'The largest change between two days was {"${:.2f}".format(data[largest_change_index][4] - data[largest_change_index][1])} on {data[largest_change_index][0]}.')

The largest change between two days was $-1.69 on 2017-08-09.


## 6. What was the average daily trading volume during this year?

In [12]:
num_days = len(data)
total_trading_volume = 0

for row in data:
    total_trading_volume += row[6]
    
print(f'The average daily trading volume was {"{:.3f}".format(total_trading_volume / num_days)}')

The average daily trading volume was 89124.337


## 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [13]:
import math

def find_median(l):
    l.sort()
    list_length = len(l)
    if list_length % 2 == 0:
        return (l[math.floor(list_length / 2)] + l[math.floor(list_length / 2) - 1]) / 2
    else:
        return l[math.floor(list_length / 2)]

trading_list = []
for row in data:
    trading_list.append(row[6])

median_trading_volume = find_median(trading_list)
print(f"The median trading volume during the year was {median_trading_volume}")

The median trading volume during the year was 76286.0
